In [1]:
import torch
import sys 


# TensorBoard(1)
from torch.utils.tensorboard import SummaryWriter
# writer=SummaryWriter('runs/mnist')
writer=SummaryWriter('runs/test')

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()

        # output=x*wT+bias [Linear function]
        self.l1=nn.Linear(input_size,hidden_size)

        # Activation Function
        self.relu=nn.ReLU()

        self.l2 = nn.Linear(hidden_size,num_classes)

        # Activation Function (Sigmoid) bec classification Problem but we will use cross entropy 
        # that apply that function inside it so we won't define that layer
    
    def forward(self,x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        return out
    
model= NeuralNet(input_size,hidden_size,num_classes).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss() # Apply Sigmoid inside it :D
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


# TensorBoard(3) Model Graph (Network)
writer.add_graph(model,samples.reshape(-1,28*28).to(device))
writer.close() #to flush


# Training
running_loss=0.0 #  To draw in tensor Board
running_correct=0 # To draw in tensor Board

n_total_steps=len(train_loader)
for epoch in range(num_epochs):
    for i , (images,labels) in enumerate(train_loader):
        # Images are 100*1*28*28
        # Reshape to 100*784
        images=images.reshape(-1,28*28).to(device) # [100,784]
        labels=labels.to(device) # [100]
  
        # Forward
        outputs=model(images)
        loss=criterion(outputs,labels)

        # Backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # For Tensor Board
        running_loss+=loss.item()
        _,predicted=torch.max(outputs.data,1)
        running_correct += (predicted == labels).sum().item()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps} , loss = {loss.item()} ')
            
            # TensorBoard(4) To add Scalar for the loss to draw training Graph
            writer.add_scalar('training loss',running_loss/100,epoch*n_total_steps + i) #/100 bec we are running every 100 step
            writer.add_scalar('accuracy',running_correct/100,epoch*n_total_steps + i) #/100 bec we are running every 100 step
            running_loss=0.0
            running_correct=0
